In [1]:
import os
import pandas as pd
import numpy as np
import random
from pandas import DataFrame
from datetime import datetime

workdir = "/home/corea/src/MDG_ML/MEYODG-51K"

In [2]:
# < Important note > [ Please download input data from https://mega.nz/#F!CeYGDKyS!uqkmWJ4E2XSGJp_C2VO2gg]

dataset = np.load(workdir + "//MEYODG51K_dataset.npz") # input file
ss0 = np.load(workdir + '//MEYODG51K_dataset_r2.npz') # split for training and test

In [3]:
x = dataset['x']
y = dataset['y']
# y_linear = dataset['y_lnIC50']
ss0_train = ss0['train']
ss0_test = ss0['test']

In [4]:
training_image_array, training_label_array = x[ss0_train], y[ss0_train]
test_image_array, test_label_array = x[ss0_test], y[ss0_test]

# # In[9]:
# ori = training_image_array
# bat = np.zeros((ori.shape[0],178))
# cat = np.hstack([ori,bat])
# training_image_array = cat

# # In[8]:
# training_image_array.shape

# # In[10]:
# ori2 = test_image_array
# bat2 = np.zeros((ori2.shape[0],178))
# cat2 = np.hstack([ori2,bat2])
# test_image_array = cat2


In [5]:
# In[15]:
ab =[]
for i in range(100,200):
    ab.append(len(training_image_array) % i)
    
print(min(ab), ab.index(min(ab)))

0 27


In [6]:
x.shape

(51787, 79724)

In [7]:
training_image_array.shape

(46609, 79724)

In [8]:
train_X, train_y, test_X, test_y = training_image_array, training_label_array, test_image_array, test_label_array

# if K.image_data_format() == 'channels_first':
#     train_X = train_X.reshape(train_X.shape[0], 1, img_rows, img_cols)
#     test_X = test_X.reshape(x_test.shape[0], 1, img_rows, img_cols)
#     input_shape = (1, img_rows, img_cols)
# else:
#     train_X = train_X.reshape(train_X.shape[0], img_rows, img_cols, 1)
#     test_X = test_X.reshape(test_X.shape[0], img_rows, img_cols, 1)
#     input_shape = (img_rows, img_cols, 1)


In [9]:
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(46609, 79724) (46609,) (5178, 79724) (5178,)


In [10]:
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
# train_X /= 255
# test_X /= 255
print('train_X shape:', train_X.shape)
print(train_X.shape[0], 'train samples')
print(test_X.shape[0], 'test samples')

train_X shape: (46609, 79724)
46609 train samples
5178 test samples


In [11]:
StartTime8 = datetime.now()
print("StartTime :", StartTime8)
import xgboost as xgb
model = xgb.XGBRegressor()
model.fit(train_X,train_y)
EndTime8 = datetime.now()
print("EndTime :", EndTime8)

StartTime : 2022-02-09 12:41:31.954850
[12:41:49] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
EndTime : 2022-02-09 15:25:02.112771


In [12]:
test_eval = model.score(test_X,test_y)

In [13]:
test_eval

0.7768190603547874

In [14]:
predicted_classes = model.predict(test_X)

In [15]:
predicted_value = predicted_classes

In [18]:
import pickle
import joblib
workdir = "/home/corea/src/MDG_ML/result_Rev/MEYODG-51K/XGBoost"
joblib.dump(model, workdir + '/MEYODG51K_r2_XGB_model.pkl') 


['/home/corea/src/MDG_ML/result_Rev/MEYODG-51K/XGBoost/MEYODG51K_r2_XGB_model.pkl']

In [19]:
a = pd.DataFrame(predicted_value)
b = pd.DataFrame(test_label_array)
c = pd.concat([a,b], axis=1)
c.columns=["Predicted","Test"]

In [20]:
c.to_csv(workdir + '//MEYODG51K_r2_XGB_pred_obs_table.csv')

In [21]:
predicted_value.shape

(5178,)

In [22]:
from scipy.stats import linregress
linregress(b[0], a[0])

LinregressResult(slope=0.7083094495037844, intercept=0.5700947839790895, rvalue=0.8855368370230131, pvalue=0.0, stderr=0.005164991986571906)

In [23]:
from sklearn.metrics import r2_score
r2_value = r2_score(b,a)
print(r2_value)

0.7768190603547874


In [24]:
rse = ((b[0]-a[0])**2).sum()
mse = rse / len(b)
print("Final rmse value is =",np.sqrt(mse))

Final rmse value is = 1.3273189210274658


In [25]:
workdir = "/home/corea/src/MDG_ML/result_Rev/MEYODG-51K"
import seaborn as sns

g = sns.jointplot(x="Test", y="Predicted", data=c, kind='reg', color='#7C7C7C',
                  joint_kws={'line_kws':{'color':'black'}})
g.savefig(workdir+ '//MEYODG51K_r2_XGB.png', dpi=300)

/home/corea/miniconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval
